Please note that we are stealing this aero model wholesale from OpenRocket with very little understanding of what is going on. Please read chapter 3 of the OpenRocket technical documentation.

This is the Barrowman method, as extended by Galejs.

# Terminology

* Angle of attack: displacement of rocket's z-axis from relative velocity vector

* Normal Force: Corrective force normal to z-axis of rocket
* Drag: Parallel to velocity (relative to medium)

In general, the normal and drag forces are not orthogonal. They are only when angle of attack is 0. We define two other forces to produce distinct pairs of orthogonal forces.

* Axial Drag: Orthogonal to Normal force, parallel to z-axis of rocket
* Side force: Orthogonal to drag, normal to direction of motion

* Roll: around z-axis of rocket
* Pitch: defined on plane of angle of attack

# Conventions
* OpenRocket defines coordinates starting at nose-tip with positive x-axis down axis.
* I define coordinates starting at base with positive z-axis up axis.

A consequence of Barrowman's thesis is that cylindrical body components do not effect normal forces or the center of pressure location.Also, for the cylinder, the cross-sectional area is not changing w.r.t. axial position, so there is no contribution to pitching moment either. However, an extension to the method shows that cylinders contribute to body lift forces when angle of attack is not small (i.e., ignored).

We have at least two fins and they aren't canted, so we have no yawing or rolling moments from the fins.

We ignore fins interference with body but calculate body's interference with fins.

We ignore pitch damping torque since it is only relevant at apogee.

Our fins are not canted, so there is no roll forcing torque.

We have done most of our calculations for Normal Forces, which are paired with Axial Drag. It will be useful for simulation purposes to break Normal Forces into components and calculate Drag forces as well which are distinct, and describe deceleration.

Barrowman gives methods for both fully turbulent and partially-laminar layers, but OpenRocket assumes boundary layers are fully turbulent (due to < 5% difference in apogee).

In [2]:
import scipy
import numpy as np

class AeroModel():
    def __init__(self):
        # kinematic viscosity of air
        self.mu = 1.5 * 10**-5 # m^2/s
        self.R_crit = 5*10**5 # critical reynolds number at which flow becomes turbulent
        self.sincAOA = 1.0 # to avoid avoiding dividing by zero, if we need it
        self.polished_surface = 0.5 # nano m
        self.painted_surface = 5 # nano m
        self.reference_len = .29591 # m, diameter
        self.A_ref = np.pi* self.reference_len**2 / 4
        self.R_crit_skin = 51 * (self.painted_surface/self.reference_len)**-1.039
        self.MACspan = self.MAC_span(height, root, tip)
        self.finAR = self.fin_aspect(height, fin_area)
        self.fin_poly = self.coefficients(self.finAR)
        self.fin_interps = self.Kvalues()
        self.bodytail = self.bodytail_interference(height, self.reference_len/2)
        self.finshape = self.roll_damp_fin_shape(height, root, tip, self.reference_len/2)
        self.CDfric_coef = frictcoef(f_B, fin_t, MAC, SA_B, SA_fin)

    #for trapezoid fins with tip parallel to root, duh
    def MAC_span(self, height, root, tip):
        return (height/3) * (root + 2*tip)/(root + tip)
    
    def Z_t(self, sweep, height): # deprecated?
        return np.tan(sweep)/height
    def CP_Z(self, z_t, root, tip): # assuming CP constant # deprecated?
        return ((z_t/3)*(root + 2*tip) + (root**2 + tip**2 + root*tip)/6)/(root + tip)

    def fin_aspect(self, height, area):
        return 2*height**2/area

    #fin interpolator, yikes copypasta
    def coefficients(self, ar):
        denom = (1- 3.4641*ar)**2
        poly=[]
        poly.append((9.16049 * (-0.588838 + ar) * (-0.20624 + ar)) / denom)
        poly.append((-31.6049 * (-0.705375 + ar) * (-0.198476 + ar)) / denom)
        poly.append((55.3086 * (-0.711482 + ar) * (-0.196772 + ar)) / denom)
        poly.append((-39.5062 * (-0.72074 + ar) * (-0.194245 + ar)) / denom)
        poly.append((12.8395 * (-0.725688 + ar) * (-0.19292 + ar)) / denom)
        poly.append((-1.58025 * (-0.728769 + ar) * (-0.192105 + ar)) / denom)
        return poly

    # coefficients for single fin supersonic normal force coefficient derivative
    # we will follow their approach and precompute some coefficients
    # and we will return linear interpolators 
    def Kvalues(self):
        gamma = 1.4 # wtf is this
        Ma0 = 1.5
        def K1(beta):
            return 2.0/beta
        def K2(beta, mach):
            return ((gamma + 1) * mach**4 - 4 * beta**2) / (4 * beta**4)
        def K3(beta, mach):
            return ((gamma + 1) * mach**8 + (
                    2 * gamma**2 - 7 * gamma - 5) * mach**6
                    + 10 * (gamma + 1) * mach**4 + 8) / (6 * beta**7)
        Ma, k1, k2, k3 = [Ma0], [], [], []
        while Ma[-1] < 5.0:
            beta = self.Beta(Ma[-1])
            k1.append(K1(beta))
            k2.append(K2(beta, Ma[-1]))
            k3.append(K3(beta, Ma[-1]))
            Ma.append(Ma[-1] + 0.1)
        trash = Ma.pop()
        k1_intrp = lambda M: np.interp(M, Ma, k1)
        k2_intrp = lambda M: np.interp(M, Ma, k2)
        k3_intrp = lambda M: np.interp(M, Ma, k3)
        return (k1_intrp, k1_intrp, k1_intrp)
    
    #fin-body interference coefficient
    def bodytail_interference(self, height, body_radius):
        return 1 + body_radius/(height + body_radius)

    # subsonic fin shape term, eq 3.70
    def roll_damp_fin_shape(self, height, root, tip, radius):
        term1 = (root + tip) * height *radius**2 / 2
        term2 = (root + 2*tip) * height**2 * radius/3
        term3 = (root + 3*tip) * height**3 / 12
        return term1+term2+term3

    # skin friction drag coefficient
    # func of body fineness, fin thickness, mean aero chord, wet area of body, wet area of fins (both sides incl)
    def frictcoef(self, f_B, fin_t, MAC, SA_B, SA_fin):
        return ((1 + 1/(2*f_B))* SA_B +
                      (1 + 2*fin_t/MAC)* SA_fin) / self.A_ref
    
    # angle of attack expects relative velocity in body coords
    def AoA(self, v_body):
        return np.arccos(np.dot(v_body/np.linalg.norm(v_body), np.array([0,0,1])))

    # reciprocal of Prandtl factor P "which corrects subsonic force coefficients for compressible flow"
    def Beta(self, mach):
        return np.sqrt(abs(mach**2 - 1))

    # robert galejs' correction term for cylindrical body section lift
    # reference area is cross-section, planform is diam*length
    # actually this comes from pg 3-11 of Fluid Dynamic Drag, Hoerner
    # ugh so confusing, do we multiply by q and A_ref to get Normal Force?
    def C_N_lift(self, body_len, alpha):
        K = 1.1 # fudge factor
        return K*body_len*self.reference_len/self.A_ref * np.sin(alpha)**2

    #Tactical Missile Design by Fleeman, ignore variance with mach
    # this is from slender body and crossflow theories # deprecated?
    def body_AeroCenter(self, alpha, total_len, nose_len):
        sin2a = np.sin(alpha)**2
        return 0.63*nose_len*(1-sin2a) + 0.5*total_len*sin2a

    # correction for CP changing with speed
    # for mach above 2, and below is interpolated
    # by a 5th order overfit polynomial lol. RHS = right hand side of equation
    def CP_Z_varying(self, MAC_length, mach):
        if mach <=0.7: #subsonic, 1/4 of MAC
            RHS = 0.25
        if mach >= 2: #supersonic, 1/2 of mac
            beta = self.Beta(mach)
            RHS = (self.finAR*beta-0.67)/(2*self.finAR*beta-1)
        else: #transonic, oh god
            x=1.0
            RHS=0
            for coeff in self.fin_poly:
                RHS += coeff*x
                x *= mach
        return MAC_length*RHS

    # thin airfoil theory of potential flow corrected for compressible flow
    def C_N_alpha_0(self, beta):
        return 2*np.pi/beta

    # single fin subsonic normal force coefficient derivative
    def C_N_alpha_1_sub(self, beta, height, midchord_sweep):
        return 2*np.pi*height**2/self.A_ref/(1+np.sqrt(
            1 + (height**2*beta/(self.A_fin*np.cos(midchord_sweep)))**2))

    # single fin supersonic normal force coefficient derivative
    # Barrowman used a third-order (!) expansion of Busemann theory
    # openrocket simplified his method by assuming fins are flat plates
    # and ignoring correction of fin-tip mach cones
    def C_N_alpha_1_super(self, alpha, mach):
        return self.A_fin/self.A_ref * (self.fin_interps[0](mach)
                              + self.fin_interps[1](mach) * alpha
                              + self.fin_interps[2](mach) * alpha**2)

    # black magic, don't ask
    # some of this could be precalced
    def C_N_alpha_1_trans(self, height, midchord_sweep,
                          alpha, mach):
        subsonic = 0.9
        supersonic = 1.5
        supersonic_b = (supersonic**2 - 1)**1.5
        sq = np.sqrt(1 + (1 - subsonic**2) * (height**2 / (self.A_fin * np.cos(midchord_sweep)))**2)
        subV = 2 * np.pi * height**2 / self.A_ref / (1 + sq)
        subD = 2 * mach * np.pi * height**6 / ((self.A_fin * np.cos(midchord_sweep))**2 
                                             * self.A_ref * sq * (1 + sq)**2)
        superV = C_N_alpha_1_super(alpha, supersonic)
        superD = - self.A_fin/self.A_ref * 2 * supersonic / supersonic_b

        trans_interp = scipy.interpolate.CubicHermiteSpline([subsonic, supersonic],
                                             [subV, superV],
                                             [subD, superD],
                                             extrapolate=False)
        return trans_interp.__call__(mach)

    # N=4 fins equally spaced, N/2
    def C_N_alpha_fins(self, height, midchord_sweep,
                       alpha, mach):
        if mach < 0.9:
            CNa1 = C_N_alpha_1_sub(self.Beta(mach), height, midchord_sweep)
        elif mach > 1.5:
            CNa1 = C_N_alpha_1_super(alpha, mach)
        else: #cry
            CNa1 = C_N_alpha_1_trans(height, midchord_sweep,
                         alpha, mach)
        return CNa1 * 2

    #fin-body interference remember to use constant
    def C_N_alpha_tail(self, CNa_fins):
        return CNa_fins * self.bodytail

    
    # subsonic roll damping from 4 fins, eq 3.69
    # v0 is free-stream velocity, omega is roll rate
    def C_ld_subsonic(self, CNa0, v0, omega):
        return 4*CNa0*omega*self.finshape/(self.A_ref*self.reference_len*v0)

    def C_ld_supersonic(self, mach, omega, v0,
                        height, root, tip):
        # local pressure coefficient of a strip of fin
        def C_P_i(eta, K):
            return (K[0] * eta
                    + K[1] * eta**2
                    + K[2] * eta**3)
        def eta_i(distance, coeff):# eq 3.62, small angle approx
            return distance*coeff # reasonable up to 20 deg
        def width(y): #trapezoid, y is height
            return y*tip/height + (height-y)*base/height

        vel_coeff = omega/v0
        eval_K = [k(mach) for k in self.fin_interps]
        mult = 4/(self.A_ref*self.reference_len)

        num_terms = 5
        del_y = height/num_terms

        terms = sum([C_P_i(eta_i(
                                i*del_y, vel_coeff),
                           eval_K) *i*del_y * width(i*del_y) * del_y
            for i in range(num_terms+1)])
        return mult*terms

    # "v0 is the free-stream velocity of the rocket, ν is the kinematic viscosity of air"
    def Reynold(self, v0):
        return self.reference_len*v0/self.mu

    def inv_Reynold(self, v0, mu): # get critical distance from nosetip # deprecated?
        return self.R_crit * self.mu / v0

    # skin friction coefficient function of reynolds number and mach
    # turbulent flow, in BarrowmanCalculator.java there is a continuous interpolation of correction around Mach 1
    # but that probably isn't needed here. could precalc a little here?
    def C_f(self, R, M):
        if M <= 1:# compressibility correction
            correction = 1 - 0.1*M**2
        elif M > 1:
            correction = (1 + 0.15 * M**2)**-0.58
        if R < 10**4: # velocity < 1 m/s
            return 1.48 * 10**-2
        elif R < self.R_crit_skin: #assuming smooth surface
            return correction * (1.5* np.log(R) - 5.6)**-2 
        else: # simplified form of eqs (3.80) and (3.79)
            if M > 1: correction = max(1/(1+ 0.18 * M**2), correction)
            return correction * 0.06821 * self.R_crit_skin**-.1925

    # skin friction drag coefficient
    # func of body fineness, fin thickness, mean aero chord, wet area of body, wet area of fins (both sides incl)
    def C_D_frict(self, Cfc):
        return Cfc * self.CDfric_coef

    # pressure drag coefficient
    # after inspecting graph of pressure drag vs mach in our openrocket model
    # i decided to simply approximate the shape wth an s-curve, since our nosecone and fins are not going to be
    # changing over the course of an optimization. coefficients from mycurvefit.com
    def C_D_press(self, M):
        return .1005025 + (.002503454 - .1005025)/(1+ (M/1.065884)**6.060682)

    # base drag coefficient
    # i am approximating this as well for simplicity, since geometry is relatively fixed
    # graph in openrocket appears piecewise continuous with a discontinuity at mach 1, of course
    # obviously if geometry of fins or nosecones change, we'll rethink this approach
    def C_D_base(self, M):
        if M <=1:
            return 0.12 - .01*M + 0.14*M**2
        else:
            return .4001341 - .1722904*M + .02131938*M**2
    # note we are ignoring parasitic drag

    # drag coefficient at zero angle of attack
    def C_D_0(self, R, M, f_B, fin_t, MAC, SA_B, SA_fin, A_ref):
        Cfc = C_f(R, M)
        CD_f = C_D_frict(Cfc, f_B, fin_t, MAC, SA_B, SA_fin, A_ref)
        CD_p = C_D_press(M)
        CD_b = C_D_base(M)
        return CD_f + CD_p + CD_b

    # drag coefficient adjusted for angle of attack
    def C_D_axial(self, alpha, CD0):
        if alpha < 0.01: return CD0
        angle = 17 * np.pi / 180
        if alpha <= angle:
            start = (0,1)
            end = (angle, 1.3)
        else:
            start = (angle, 1.3)
            end = (np.pi/2, 0)
        CD_interp = scipy.interpolate.CubicHermiteSpline([start[0], end[0]],
                                             [start[1], end[1]],
                                             [0, 0],
                                             extrapolate=False)
        return CD_interp.__call__(alpha)